In [1]:
# a very simple Flask based web API for production prototype
# run this cell to start the web server

import flask
import pandas as pd
import numpy as np
import dill
import json

from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion, make_union
from sklearn.base import TransformerMixin, BaseEstimator

from sklearn.preprocessing import StandardScaler, Imputer  # import imputer and scaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV

from dm_tools import data_prep

required_columns = ['GiftCnt36', 'GiftCntAll',
                    'GiftCntCard36', 'GiftCntCardAll', 'GiftAvgLast',
                    'GiftAvg36', 'GiftAvgAll', 'GiftAvgCard36', 'GiftTimeLast',
                    'GiftTimeFirst', 'PromCnt12', 'PromCnt36', 'PromCntAll',
                    'PromCntCard12', 'PromCntCard36', 'PromCntCardAll', 'StatusCat96NK',
                    'StatusCatStarAll', 'DemCluster', 'DemAge', 'DemGender', 'DemHomeOwner',
                    'DemMedHomeValue', 'DemPctVeterans', 'DemMedIncome']

app = flask.Flask(__name__)
dilled_model = dill.load(file=open('model.pkl', 'rb'))

@app.route('/predict', methods = ['POST'])
def predict():
    input_data = flask.request.json
    df = pd.read_json(input_data)
    
    # verify if the received dataset is complete
    for col in required_columns:
        if col not in df.columns:
            return 'Incomplete columns', 400  # return a bad request
    
    prediction_values = dilled_model.predict(df)
    return json.dumps(prediction_values.tolist()), 200

if __name__ == '__main__':
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/May/2018 02:05:15] "POST /predict HTTP/1.1" 200 -
